In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime as dt
from selenium.common.exceptions import StaleElementReferenceException
import re
from bs4 import BeautifulSoup

In [2]:
url = 'https://line4bet.ru/1x-13-03-2023-football/'

In [3]:
driver = webdriver.Chrome()
driver.get(url)
assert 'Движение линий' in driver.title

In [4]:
# "Теннис" button searching
tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
assert len(tennis_btn) > 0
tennis_btn[0].click()

In [5]:
# looking for available date
previous_date = dt.datetime.today() - dt.timedelta(days=1)
prev_day = previous_date.day
dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
try:
    dates_objects[prev_day-1].click()
except:
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    dates_objects[prev_day-1].click()

In [21]:
def get_data_event(event):
    cleaned_event = re.sub('<tbody>|<tr>|<td>|</tbody>|</tr>|</td>|<td colspan="2">|\t','', event)
    nsymbols_data = 16
    date = cleaned_event[:nsymbols_data]
    cleaned_event_ = cleaned_event[nsymbols_data:].strip()
    splitted_event = cleaned_event_.split(';')
    players = splitted_event[0]
    cleaned_event_ = splitted_event[1].strip()
    if 'Победа' in cleaned_event:
        winner = cleaned_event_.split(', ')[1].split(',')[0].strip()
        splitted_event = cleaned_event_.split(', ')[0].split('  ')
        event_ = ' '.join(splitted_event[1:3])
        score = splitted_event[-1]
    else:
        winner = ''
        event_ = ' '.join(cleaned_event_.split('  ')[1:3])
        score = ''
    return date, event_, players, winner, score


def get_data_table(tables):
    page_tables = []
    for str_table in tables:
        soup = BeautifulSoup(str_table, 'html.parser')
        cleaned_table_data = [
            re.sub('<tr> | </tr>| class="c1"| class="c2"', '', str(x)) for x in soup.find_all('tr')[1:-1]
        ]
        columns = soup.find_all('th')
        columns = [re.sub('<th> | </th>', '', str(x)) for x in columns]
        table = []
        for str_ in cleaned_table_data: 
            soup_str = BeautifulSoup(str_, 'html.parser')
            table.append([x.text for x in soup_str.find_all('td')])
        table = pd.DataFrame(data=table, columns=columns)
        page_tables.append(table)
    return page_tables


def restart_page(driver):
    tennis_btn = driver.find_elements(By.ID, 'tennis_btn')
    tennis_btn[0].click()
    # looking for available date
    previous_date = dt.datetime.today() - dt.timedelta(days=1)
    prev_day = previous_date.day
    dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
    try:
        dates_objects[prev_day-1].click()
    except:
        dates_objects = driver.find_elements(By.XPATH, "//td[@Data-handler]")
        dates_objects[prev_day-1].click()

In [22]:
data_events = []
data_tables = []
pressed_tags = []
tbodys = []
npages = driver.find_elements(By.CLASS_NAME, 'pages')
act_p = []
# pages iterating
for i in range(len(npages)):
    try:
        driver.execute_script("arguments[0].click();", npages[i])
    except StaleElementReferenceException:
        npages = driver.find_elements(By.CLASS_NAME, 'pages')
        driver.execute_script("arguments[0].click();", npages[i])    
    html_source_code = driver.execute_script("return document.body.innerHTML;")
    html_soup = BeautifulSoup(html_source_code, 'html.parser')
    tbody_data = np.array([str(x) for x in html_soup.find_all(["tbody"])], dtype='object')
    tbodys.append(tbody_data)
    # events parsing
    events_indexes = ['<tbody><tr><td>' in x and 'colspan="2"' in x for x in tbody_data]
    events = tbody_data[events_indexes]
    data_events.append([get_data_event(x) for x in events])
    # data_tables
    tables = tbody_data[['Дата-время скана линии' in x for x in tbody_data]]
    data_tables.append(get_data_table(tables))

In [26]:
# incorrect
[len(data_tables[i]) for i in range(len(data_tables))]

[50, 50, 55]

correct -> 50, 55, 42

In [35]:
# incorrect

In [29]:
data_tables[0][0]

,Дата-время скана линии,П1,П2,1Х,2Х,Х,12,ТМ,ТМ бол,ТМ мен,...,Коэф Ф1,ФОРА 2,Коэф Ф2,Инд ТОТ 1,ИТ1 бол,ИТ1 мен,Инд ТОТ 2,ИТ2 бол,ИТ2 мен,Прим.
0,23 май 02:10,"2,49","1,54",,,,,"22,5","2,02","1,78",...,2,"-2,5","1,8","11,5","1,72","2,05","12,5","1,85","1,89",
1,23 май 12:50,"2,49","1,54",,,,,"22,5","1,98","1,81",...,"1,99","-2,5","1,81","11,5","1,7","2,08","12,5","1,82","1,92",
2,23 май 14:10,"2,52","1,53",,,,,"22,5","1,98","1,81",...,2,"-2,5","1,8","11,5","1,7","2,08","12,5","1,81","1,94",
3,23 май 14:20,"2,51","1,54",,,,,"22,5","1,91","1,91",...,"1,97","-2,5","1,86","11,5","1,71","2,16","12,5","1,86","1,97",


In [33]:
data_tables[1][0]

,Дата-время скана линии,П1,П2,1Х,2Х,Х,12,ТМ,ТМ бол,ТМ мен,...,Коэф Ф1,ФОРА 2,Коэф Ф2,Инд ТОТ 1,ИТ1 бол,ИТ1 мен,Инд ТОТ 2,ИТ2 бол,ИТ2 мен,Прим.
0,23 май 02:10,"2,49","1,54",,,,,"22,5","2,02","1,78",...,2,"-2,5","1,8","11,5","1,72","2,05","12,5","1,85","1,89",
1,23 май 12:50,"2,49","1,54",,,,,"22,5","1,98","1,81",...,"1,99","-2,5","1,81","11,5","1,7","2,08","12,5","1,82","1,92",
2,23 май 14:10,"2,52","1,53",,,,,"22,5","1,98","1,81",...,2,"-2,5","1,8","11,5","1,7","2,08","12,5","1,81","1,94",
3,23 май 14:20,"2,51","1,54",,,,,"22,5","1,91","1,91",...,"1,97","-2,5","1,86","11,5","1,71","2,16","12,5","1,86","1,97",


In [34]:
data_tables[2][0]

,Дата-время скана линии,П1,П2,1Х,2Х,Х,12,ТМ,ТМ бол,ТМ мен,...,Коэф Ф1,ФОРА 2,Коэф Ф2,Инд ТОТ 1,ИТ1 бол,ИТ1 мен,Инд ТОТ 2,ИТ2 бол,ИТ2 мен,Прим.
0,23 май 15:00,"4,36","1,18",,,,,"19,5","1,89","1,81",...,"1,85","-5,5","1,85",,,,,,,
1,23 май 15:30,"4,36","1,18",,,,,"19,5","1,9","1,8",...,"1,86","-5,5","1,84",,,,,,,
